In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
import os
import torch
from PIL import Image

In [3]:
class ImageFolderWithLabels(Dataset):
    def __init__(self, root_dir, transform=None, half_size=False):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self._get_image_paths_and_labels(half_size)

    def _get_image_paths_and_labels(self, half_size=False):
      for subfolder in ["real", "fake"]:
            subfolder_path = os.path.join(self.root_dir, subfolder)
            image_list = os.listdir(subfolder_path)  # Get all filenames

            if half_size:
                image_list = image_list[::2]  # Select every alternate image

            for filename in image_list:
                if filename.endswith(".jpg"):
                    img_path = os.path.join(subfolder_path, filename)
                    self.image_paths.append(img_path)
                    self.labels.append(0 if subfolder == "real" else 1)  # Assign label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)

        return img, label

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),  # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize using ImageNet statistics
])

In [5]:
# Load data from each directory
train_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/faceforensics_processed_split', 'Train'), transform=transform)#, half_size=True)
test_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/faceforensics_processed_split', 'Test'), transform=transform)#, half_size=True)
val_data = ImageFolderWithLabels(os.path.join('/content/drive/MyDrive/faceforensics_processed_split', 'val'), transform=transform)#, half_size=True)

In [6]:
print(len(train_data))
print(len(test_data))
print(len(val_data))

2048
255
257


In [7]:
BATCH_SIZE=32

In [8]:
# Create dataloaders for training, testing, and validation
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)  # Adjust batch size as needed
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
convnext = torchvision.models.convnext_tiny(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/convnext_tiny-983f1562.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny-983f1562.pth
100%|██████████| 109M/109M [00:02<00:00, 44.9MB/s]


In [10]:
total_params = sum(p.numel() for p in convnext.parameters())
total_params

28589128

In [11]:
import torch.optim as optim
optimizer = optim.Adam(convnext.parameters(), lr=0.001)

In [12]:
# Define your loss function
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

In [13]:
# Define the device to run the model on (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [14]:
convnext.to(device)

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96, out_features=384, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=384, out_features=96, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=

In [15]:
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=8):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        # Training phase
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Optimize
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        # Calculate training loss and accuracy
        epoch_train_loss = running_loss / len(train_loader.dataset)
        epoch_train_acc = correct / total

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_running_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs = model(val_inputs)
                val_loss = criterion(val_outputs, val_labels)
                val_running_loss += val_loss.item() * val_inputs.size(0)
                _, val_predicted = val_outputs.max(1)
                val_total += val_labels.size(0)
                val_correct += val_predicted.eq(val_labels).sum().item()

        # Calculate validation loss and accuracy
        epoch_val_loss = val_running_loss / len(val_loader.dataset)
        epoch_val_acc = val_correct / val_total

        # Print epoch statistics
        print(f"Epoch {epoch + 1}/{num_epochs}",
              f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.4f}",
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    print('Training complete')


num_epochs = 10


train_model(convnext, train_loader, val_loader, criterion, optimizer, device, num_epochs)


Epoch 1/10 Train Loss: 0.9503, Train Acc: 0.5747 Val Loss: 0.5792, Val Acc: 0.7626
Epoch 2/10 Train Loss: 0.5220, Train Acc: 0.7510 Val Loss: 0.5101, Val Acc: 0.7393
Epoch 3/10 Train Loss: 0.4630, Train Acc: 0.7837 Val Loss: 0.3663, Val Acc: 0.8405
Epoch 4/10 Train Loss: 0.3836, Train Acc: 0.8413 Val Loss: 0.4754, Val Acc: 0.8016
Epoch 5/10 Train Loss: 0.3735, Train Acc: 0.8560 Val Loss: 0.3001, Val Acc: 0.8988
Epoch 6/10 Train Loss: 0.3607, Train Acc: 0.8638 Val Loss: 0.1917, Val Acc: 0.9494
Epoch 7/10 Train Loss: 0.2806, Train Acc: 0.8906 Val Loss: 0.1973, Val Acc: 0.9300
Epoch 8/10 Train Loss: 0.2397, Train Acc: 0.9106 Val Loss: 0.1717, Val Acc: 0.9300
Epoch 9/10 Train Loss: 0.2582, Train Acc: 0.9087 Val Loss: 0.1824, Val Acc: 0.9339
Epoch 10/10 Train Loss: 0.1962, Train Acc: 0.9321 Val Loss: 0.1223, Val Acc: 0.9611
Training complete


In [16]:
def test_model(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # No need to calculate gradients during testing
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate the loss
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss = running_loss / len(test_loader.dataset)
    test_acc = correct / total
    print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}")


In [17]:
test_model(convnext, test_loader, criterion, device)

Test Loss: 0.1736, Test Accuracy: 0.9412


In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_model(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    confusion_mat = confusion_matrix(true_labels, predictions)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(confusion_mat)

#Assuming you have already defined your test_loader and model
evaluate_model(convnext, test_loader, device)



Accuracy: 0.9412
Precision: 0.9825
Recall: 0.8960
F1 Score: 0.9372
Confusion Matrix:
[[128   2]
 [ 13 112]]
